1. 정류장 노선 이름을 순서대로 리스트 저장
2. 정류장 첫차, 막차, 배차간격 리스트 저장
3. 길찾기 > 순서대로 정류장 시간 딕셔너리 저장
4. 버스별 데이터프레임 만들고 엑셀 저장 
  column    노선도   첫차   배차간격에따른차고지출발시각(총 횟수에 따라 설정)          
   시각
5. 목적지 두 개와 시간을 찍었을 때
 - 각 데이터프레임에서 최단 버스 시각 출력 
6. 목적지 세 개(A-B-C)를 찍었을 때
 - A->B->C, A->C->B, 
   B->A->C, B->C->A,
   C->A->B, C->B->A
    > 목적지별 데이터프레임에서 매칭되는 '정류장'을 중간에 넣기
     --> 4개 이상 목적지에서 최단 경로가 될 수 있음  
 - 목적지별 1시간 관람으로 가정하여 배차시간 컬럼 가져오기

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs

In [2]:
# 20230705171919_관광지별 성_연령 발문 집중률 조회.csv
place = ['경주골굴사', '경주김유신묘', '경주동궁과월지'
,'경주동부사적지대','경주무열왕릉','경주문무대왕릉'
,'경주불국사', '경주석굴암', '경주신문왕릉', '경주월성(반월성)'
, '경주첨성대', '경주금곡사', '경주기림사', '경주백률사', '경주분황사' 
,'경주연지암', '경주주사암']

In [3]:
path='../../driver/chromedriver.exe'
driver = webdriver.Chrome(service=Service(path))

### 노선도 가져오기

In [4]:
def bus_route(driver):
    lst = []
    bus_list = []
    route = driver.find_element(By.CLASS_NAME, "route_area").text
    lst.append(route.split(sep='\n'))
    lst = lst[0]
    for i in range(len(lst)):
        bus_list.append(lst[3::3])
    time.sleep(6)
    return lst

### 버스 정보 가져오기

In [5]:
def bus_info(driver):
    # 버스 번호 
    bus_num = driver.find_element(By.CLASS_NAME,'title_box').text[2:]

    # 운행 첫 차 시각
    bus_first_time_text = driver.find_element(By.CLASS_NAME,"item_bus_info.ng-star-inserted").text
    bus_first_time = re.sub(r'[^0-9]', '',bus_first_time_text)
    time.sleep(4)

#     # 배차간격
#     bus_term = driver.find_element(By.XPATH,"""/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/directions-layout/directions-result/div[2]/directions-details-result/entry-layout/entry-bus-route/div[1]/div[2]/div/div[1]/ul/li[3]/div""").text    
    data = [bus_num+'번', bus_first_time[:4]]
    time.sleep(4)
    return data

### 다음 place 요소 가져오기

In [6]:
def next_place(driver):
    # 도착지점 지우기
    next_place = driver.find_element(By.ID, 'directionGoal1')
    next_place.clear()

### 정류장 이름 및 고유번호 가져오기

In [7]:
def bus_information(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 정류장 이름 리스트 저장
    station_name = []
    d = soup.select('div.route_area>ul>li>a>span>strong')
    for x in d:
        station_name.append(x.text)
    # station_name
    time.sleep(3)
    # 버스 고유 번호 리스트 저장
    station_number = []
    e = soup.select('div.route_area>ul>li>a>div.route_box')
    for x in e:
        station_number.append(x.text)
    time.sleep(3)
    
    bus_data = {'버스노선':station_name, '정류장번호':station_number}
    return bus_data

### 각 정류장마다 출발지~도착지 시간 구하기

### 웹 데이터 추출

In [8]:
target = len(place)

# 네이버 지도 접속
driver.get('https://map.naver.com/')
time.sleep(3)

# 배너 '길찾기 클릭'
element = driver.find_element(By.CLASS_NAME,"link_navbar.directions")
element.click()
time.sleep(2)

# 출발지 입력에 '경주시외버스터미널' 검색
dep_search = driver.find_element(By.ID, 'directionStart0')
start_searh_name = "경주시외버스터미널"
dep_search.send_keys(start_searh_name)
dep_search.send_keys(Keys.ENTER)
time.sleep(3)

for i in range(target):
    # 도착지 입력에 place 중 하나 입력
    arr_search = driver.find_element(By.ID, 'directionGoal1')
    arr_searh_name = place[i]
    arr_search.send_keys(arr_searh_name)
    arr_search.send_keys(Keys.ENTER)
    time.sleep(3)

    # '길찾기' 클릭
    element = driver.find_element(By.CLASS_NAME,"btn.btn_direction.active")
    element.click()
    time.sleep(3)

    ### 첫 번째로 보이는 최적경로 버스와 노선도 가져오기
    # 상세보기
    bus_detail = driver.find_element(By.CLASS_NAME,"btn_detail")
    bus_detail.click()
    time.sleep(3)

    # 버스 클릭
    bus_click = driver.find_element(By.CLASS_NAME,"bus_name_box")
    bus_click.click()
    time.sleep(3)
        
    # 버스 번호 저장
    bus_name = driver.find_element(By.CLASS_NAME, "title_box").text
    time.sleep(3)
    
    # 버스 배차간격, 첫차, 버스번호 불러오기
    first = driver.find_element(By.CLASS_NAME,"item_bus_info.ng-star-inserted")
    first_time = first.find_element(By.CLASS_NAME,"value").text[3:]
    time.sleep(2)

    per_time = driver.find_element(By.CLASS_NAME,"value.ng-star-inserted").text
    bus_init = {'첫차':first_time, '배차간격':per_time}
    bus_num = driver.find_element(By.CLASS_NAME, "title_wrap_box").text
    time.sleep(3)
    
    # 버스 정보 불러와서 엑셀로 저장
    df = pd.DataFrame(bus_information(driver))
    df['bus_name'] = bus_name
    df.set_index('bus_name').reset_index(inplace=True)
    df['도착시간'] = first_time
    df['배차간격'] = per_time
    df['버스번호'] = bus_num
    df.to_excel(place[i]+".xlsx")
    
    # 도착지 이전 입력값 지우기
    arr_search.send_keys(Keys.CONTROL + "A")
    arr_search.send_keys(Keys.DELETE)
    time.sleep(2)

In [12]:
df = pd.read_excel('./좌석100.xlsx')
df

,Unnamed: 0,버스노선,정류장번호,버스번호,도착시간,배차간격
0,0,시외버스.고속버스터미널,1225,좌석100,06:20,평일 20분
1,1,고속버스.시외버스터미널,1045,좌석100,06:20,평일 20분
2,2,서라벌사거리.황리단길,1190,좌석100,06:20,평일 20분
3,3,중앙시장,1309,좌석100,06:20,평일 20분
4,4,법원.동산병원 건너,1083,좌석100,06:20,평일 20분
...,...,...,...,...,...,...
95,95,법원.동산병원,1137,좌석100,06:20,평일 20분
96,96,중앙시장,1308,좌석100,06:20,평일 20분
97,97,서라벌사거리.황리단길,1189,좌석100,06:20,평일 20분
98,98,고속버스터미널,1044,좌석100,06:20,평일 20분
